In [2]:
from stable_baselines3 import A2C, PPO
from stable_baselines3.common.env_checker import check_env
from meta.custom_env import CustomEnv
from utils.fetch_data import Api, fetch_data

data = fetch_data(Api.YAHOO, 'BTC-USD', '2018-01-01', '2023-01-01')

env = CustomEnv(data, action_spece=3, observation_space=5)
check_env(env)

model = A2C("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10000, log_interval=100)

[*********************100%***********************]  1 of 1 completed


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
------------------------------------
| time/                 |          |
|    fps                | 302      |
|    iterations         | 100      |
|    time_elapsed       | 1        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -1.01    |
|    explained_variance | 0.0      |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | 0.227    |
|    value_loss         | 0.675    |
------------------------------------
-----------------------------------------
| time/                 |               |
|    fps                | 299           |
|    iterations         | 200           |
|    time_elapsed       | 3             |
|    total_timesteps    | 1000          |
| train/                |               |
|    entropy_loss       | -0.768        |
|    explained_variance | 5.9604645e-08 |
|    learn

In [ ]:
import pandas as pd
import quantstats as qs
from utils.plot_buy_and_sell import plot_buy_and_sell

test_data = fetch_data(Api.YAHOO, 'BTC-USD', '2023-01-01', '2024-01-01')

test_env = CustomEnv(test_data, action_spece=3, observation_space=5)
check_env(test_env)

portfolio_values = []

observation, info = test_env.reset()

while True:
    action, _states = model.predict(observation)

    observation, reward, terminated, truncated, info = test_env.step(action)
    done = terminated or truncated
    portfolio_values.append(info['portfolio_value'])

    if done:
        break

actions_data = test_env.close()
actions_data.index = pd.to_datetime(test_data.index)

plot_buy_and_sell(actions_data)

dates = pd.date_range(start=test_data.index[0], periods=len(portfolio_values), freq="D")
portfolio_series = pd.Series(portfolio_values, index=dates)

returns = portfolio_series.pct_change().fillna(0)

qs.reports.basic(returns)

TypeError: fetch_data() missing 1 required positional argument: 'end'